# **Iteration 3- LEGAL-BERT**
## MSc Project 
## Surya L Ramesh (0206793)

In [1]:
''' LEGAL-BERT - trained upto 20 epochs and best chosen'''
    

' LEGAL-BERT - trained upto 20 epochs and best chosen'

## **Mount  google drive - will need authorisation**

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

ModuleNotFoundError: No module named 'google.colab'

## **Install required Libraries**

In [ ]:

!pip install transformers
!pip install sentencepiece

In [ ]:
import pandas as pd
import re
import torch

from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence

import pickle
import os
from transformers import BertTokenizer  # modified
from transformers import AutoTokenizer, AutoModel
import numpy as np

import os
import pickle
import fnmatch
from termcolor import colored
import pandas as pd
import numpy as np
import re

import copy
import torch.optim as optim
import torch
import torch.nn as nn
import random
from transformers import BertForSequenceClassification, AdamW


##**Check GPU and memory availability**
   - code is from Google Colab official examples: https://colab.research.google.com/notebooks/pro.ipynb

In [ ]:
# check GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
# check Memory 

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

## **Access datasets from the mounted Google Drive**

In [ ]:
# Use Relative path 
#path = 'SNLI_Data_Csv'
path = "/content/drive/MyDrive/CleanedData_notEncoded"
modelPath = "/content/drive/MyDrive/CleanedData_notEncoded/model_22_07_LegalBert"
dirList = os.listdir(path)

In [ ]:

dirList

## **Load and view the previously cleaned dataset in Google Drive** 
-  after cleaning , only wanted columns are kept , remove'-' rows, from gold_label, remove rows with na , rename columns,

In [ ]:
for dir in dirList:
  sub_path = os.path.join(path,dir) 
  #print(sub_path)
  if fnmatch.fnmatch(sub_path,'*_train*'):
    print(colored('\n In Filename and path: %s' % (sub_path),'blue')) 
    train_tidy = pd.read_csv(sub_path,nrows =350000)  # changed to 10K for experimentation 
    
    
  if fnmatch.fnmatch(sub_path,'*_val*'):  
    print(colored('\n In Filename and path: %s' % (sub_path),'blue')) 
    val_tidy = pd.read_csv(sub_path,nrows =10000) 

  if fnmatch.fnmatch(sub_path,'*df_test.csv*'):  
    print(colored('\n In Filename and path: %s' % (sub_path),'blue')) 
    test_tidy = pd.read_csv(sub_path,nrows =10000) 

In [ ]:
test_tidy.head(5)

In [ ]:
train_tidy.head(5)

In [ ]:
val_tidy.head(5)

## **Define class for SNLIDataLegalBert**

In [ ]:
''' initialising , data Loader (batches) ,encoding ,  Add [CLS] and [SEP] tokens, join the two sentences for the train and val datasets'''

class SNLILegalBert(Dataset):

  def __init__(self, train_df, val_df):
    self.label_dict = {'entailment': 0, 'contradiction': 1, 'neutral': 2}

    self.train_df = train_df
    self.val_df = val_df

    self.base_path = '/content/'
    self.tokenizer = BertTokenizer.from_pretrained("nlpaueb/legal-bert-small-uncased", do_lower_case=True) 
    self.train_data = None
    self.val_data = None
    self.init_data()

  def init_data(self):
    
    self.train_data = self.load_data(self.train_df)
    self.val_data = self.load_data(self.val_df)

  def load_data(self, df):
    MAX_LEN = 512   
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['premise'].to_list()
    hypothesis_list = df['hypothesis'].to_list()
    label_list = df['target'].to_list()

    for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
      pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(self.label_dict[label])
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
    print(len(dataset))
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    train_loader = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    val_loader = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=batch_size
    )
    return train_loader, val_loader

In [ ]:
# convert columns of interest to string type

In [ ]:
train_tidy['premise'] = train_tidy['premise'].astype(str)
train_tidy['hypothesis'] = train_tidy['hypothesis'].astype(str)

In [ ]:
val_tidy['premise'] = val_tidy['premise'].astype(str)
val_tidy['hypothesis'] = val_tidy['hypothesis'].astype(str)

In [ ]:
test_tidy['premise'] = test_tidy['premise'].astype(str)
test_tidy['hypothesis'] = test_tidy['hypothesis'].astype(str)

In [ ]:
test_tidy['premise']

## **Setup for Training,Validating and Final Test**

### **Define class for SNLIDataLegalBertPredictor- Used for the TestSet**

In [ ]:
''' This class is for the Test set (prediction) to Initialize , data Loader (batches) ,
encoding ,  Add [CLS] and [SEP] tokens, join the two sentences '''

class SNLIBertPredictor(Dataset):

  def __init__(self,input_df):
    self.label_dict = {'entailment': 0, 'contradiction': 1, 'neutral': 2}

    self.input_df = input_df

    self.base_path = '/content/'

    self.tokenizer = BertTokenizer.from_pretrained("nlpaueb/legal-bert-small-uncased")
    
    self.input_data = None
   # self.val_data = None
    self.init_data()

  def init_data(self):
    
    self.input_data = self.load_data(self.input_df)

  def load_data(self, df):
    MAX_LEN = 512   
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['premise'].to_list()
    hypothesis_list = df['hypothesis'].to_list()
    label_list = df['target'].to_list()

    for (premise, hypothesis,label) in zip(premise_list, hypothesis_list,label_list):
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
      pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(self.label_dict[label])
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids,y)
    print(len(dataset))
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    input_loader = DataLoader(
      self.input_data,
      shuffle=shuffle,
      batch_size=batch_size
    )
    return input_loader

### **Call SNLILegalBERT on the train and val dataset**

In [ ]:

snli_dataset = SNLILegalBert(train_tidy, val_tidy)

### **Load train and val dataset**

In [ ]:
train_loader, val_loader = snli_dataset.get_data_loaders(batch_size=16)

In [ ]:

model = BertForSequenceClassification.from_pretrained("nlpaueb/legal-bert-small-uncased",num_labels=3)
model.to(device)

In [ ]:
from transformers import EarlyStoppingCallback

early stopping here, if time allows


In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [ ]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=False)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:

def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

In [ ]:
# shutil - high-level operations on files

import shutil

In [ ]:
# save model 

def save_checkpoint(state, is_best, epoch ):
   path_to_model=modelPath + '/model_LegalBert_epoch{}.pt'.format( epoch+1)
   torch.save(state, path_to_model)
   print("The model has been saved in {}".format(path_to_model))
   if is_best:
        shutil.copyfile(path_to_model, 'model_best.pth.tar')

In [ ]:
import time

EPOCHS =1 # maximum no.of epochs to run 

def train(model, train_loader, val_loader, optimizer,epoch_to_resume):  
  total_step = len(train_loader)
  best_acc = 0
  is_best = False

  # check to resume from a previously saved epoch checkpoint

  if (epoch_to_resume> 0 ):
        path_to_model= modelPath +'/model_LegalBert_epoch{}.pt'.format( epoch_to_resume)
        if os.path.isfile(path_to_model):
            print("=> loading checkpoint '{}'".format(path_to_model))
            checkpoint = torch.load(path_to_model)
            savd_epoch = checkpoint['epoch']
            best_acc = checkpoint['best_acc']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(path_to_model, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(path_to_model))

  # training and validation for each epoch

  for epoch in range(epoch_to_resume, EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0

    # for each batch 
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(train_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      
      loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()

      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)

      loss.backward()
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    # get average train accuracy and loss across all batches 
    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    model.eval()
    total_val_acc  = 0
    total_val_loss = 0

    
   
    with torch.no_grad():
      for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(val_loader):
        optimizer.zero_grad()
        pair_token_ids = pair_token_ids.to(device)
        mask_ids = mask_ids.to(device)
        seg_ids = seg_ids.to(device)
        labels = y.to(device)

        
        # prediction = model(pair_token_ids, mask_ids, seg_ids)
        loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()
        
        # loss = criterion(prediction, labels)
        acc = multi_acc(prediction, labels)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)
    if(best_acc< val_acc):
      best_acc = val_acc 
      is_best = True
    else:
      is_best = False
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)

    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    
    

    # Saving the model
    save_checkpoint({
            'epoch': epoch + 1,
            'best_acc': best_acc,
            'state_dict': model.state_dict(),
            'optimizer' : optimizer.state_dict(),
        }, is_best,epoch)
   

## **Training with the trainset and validating with validation set**

In [ ]:
# commented out once epoch is chosen 
'''- this section is for training the model on trainset and validating on validation set'''

#%% time 
#epoch_to_resume =6
#train(model, train_loader, val_loader, optimizer,epoch_to_resume)

In [ ]:
print('Done')

## **Testing chosen Epoch with TestSet**

In [ ]:
'''- this section is for testing the chosen model on Testset'''


import time


def test(model, test_loader, optimizer,epoch_to_use):  
  # check to resume from a previously saved epoch checkpoint
  if (epoch_to_use > 0 ):
        path_to_model= modelPath +'/model_LegalBert_epoch{}.pt'.format( epoch_to_resume)
        if os.path.isfile(path_to_model):
            print("=> loading checkpoint '{}'".format(path_to_model))
            checkpoint = torch.load(path_to_model)
            savd_epoch = checkpoint['epoch']
            best_acc = checkpoint['best_acc']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(path_to_model, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(path_to_model))
            return; 
  y_pred_all = np.array([])
  y_test_all = np.array([])
  start = time.time() 
  total_test_loss = 0
  total_test_acc  = 0

  for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(test_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      
      result = model(pair_token_ids, 
                            token_type_ids=seg_ids, 
                            attention_mask=mask_ids, 
                            labels=labels)
      loss = result.loss
      prediction = result.logits
      if(torch.any(prediction.isnan())):
        break;
      
      acc = multi_acc(prediction, labels)
      #Add all predictions to y_pred_arr and all lables to y_test_arr
      y_pred = torch.log_softmax(prediction, dim=1).argmax(dim=1)
      y_pred_arr =  y_pred.to("cpu").numpy().squeeze() 
      y_test_arr = labels.to("cpu").numpy().squeeze()
      y_pred_all = np.append(y_pred_all,y_pred_arr, axis=0)
      y_test_all = np.append(y_test_all,y_test_arr, axis=0)  

      total_test_loss += loss.item()
      total_test_acc  += acc.item()

      test_acc  = total_test_acc/len(test_loader)
      test_loss = total_test_loss/len(test_loader)
      end = time.time()
      hours, rem = divmod(end-start, 3600)
      minutes, seconds = divmod(rem, 60)

  print(f'With Epoch {epoch_to_use}: test_loss: {test_loss:.4f} test_acc: {test_acc:.4f}')
  print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
  return y_pred_all,y_test_all 



In [ ]:
#Create object of SNLIAlberPredictor
#GEt test dataloader
snli_testset = SNLIBertPredictor(test_tidy)
test_loader = snli_testset.get_data_loaders(batch_size=16)

In [ ]:

epoch_to_resume =10 # chosen epoch based on training and val accuracy /loss chart 

y_pred_all,y_test_all = test(model, test_loader, optimizer,epoch_to_resume)

## **Predict on Testset and Visualise Results**


In [ ]:
# Libraries for this section 

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import itertools

In [ ]:
''' function for custom confusion matrix in colour  '''

def plot_confusion_matrix_custom(cm, classes, title='Confusion matrix - LEGAL-BERT ', cmap=plt.cm.Blues):
    
    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes, rotation=45)

    fmt =  'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.grid(False) 
   
    plt.show()

In [ ]:
''' function written to print classification report and display custom confusion matrix'''


def visualize_confusion_matrix(y_pred_argmax, y_true):

    classes = ['entailment','contradiction','neutral']
    cm = confusion_matrix(y_true, y_pred_argmax)
    con_mat_df = pd.DataFrame(cm)
    
    print(classification_report(y_pred_argmax, y_true,target_names=classes))

    plot_confusion_matrix_custom(cm,classes)

In [ ]:
print(y_pred_all)
print(y_test_all)

In [ ]:
#classification report and Confusion matrix 
visualize_confusion_matrix(y_pred_all,y_test_all)

## -------------------------------    End of code for LEGAL BERT ------------------------------------------

## Testing with Legal based sentences provided by company

In [ ]:
# function to load data and tidy it 
# pass the dataframe, kepp only unwanted columns, remove'-' rows, from gold_label, remove rows with na , rename columns,lower case, replace all given special characters
# diplay head()

def data_load_tidy(dataframe): # remove isTest- not needed???
    
    dataframe = dataframe[['sentence1','sentence2','gold_label']]
    
    dataframe = dataframe[dataframe['gold_label'] != '-' ]
    dataframe= dataframe.dropna(axis=0, inplace=False) # fixed additional nan in test and val with inplace=False 
    data_tidy = dataframe.rename(columns= {'sentence1':'premise', 'sentence2':'hypothesis', 'gold_label':'target'}, inplace=False)
    #data_tidy['premise'] = data_tidy['premise'].astype(str)
  
    data_tidy['premise'] = data_tidy.premise.str.lower()
    data_tidy['hypothesis'] = data_tidy.hypothesis.str.lower().str.replace('[^a-zA-Z ]' , '')    # lower, replace all  given special characters
  
    print(colored('\n  data is tidied ....','green')) 
    return data_tidy

    

In [ ]:
for dir in dirList:
  sub_path = os.path.join(path,dir) 
 
  if fnmatch.fnmatch(sub_path,'*df_testlegal.csv*'):  
    print(colored('\n In Filename and path: %s' % (sub_path),'blue')) 
    df_legaltest = pd.read_csv(sub_path,nrows =20) 
    legaltest_tidy = data_load_tidy(df_legaltest)

In [ ]:
legaltest_tidy.head(3)

In [ ]:
legaltest_tidy.shape[0]

In [ ]:
#Create object of the legal predictor
#get legaltest set
snli_legaltestset = SNLIBertPredictor(legaltest_tidy)
legaltest_loader = snli_legaltestset.get_data_loaders(batch_size=16)

In [ ]:
# for LEGAL-BERT - testing on small Legalset

epoch_to_resume = 10
y_pred_legal,y_test_legal = test(model, legaltest_loader, optimizer,epoch_to_resume)

## Download colab notebook as HTML

In [ ]:
## download as html
%%shell
jupyter nbconvert --to html /content/Iteration3_LEGAL_BERT.ipynb